In [13]:
from datasets import load_dataset
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    DataCollatorWithPadding,
    Trainer,
    TrainingArguments,
)

dataset = load_dataset("glue", "mrpc")


tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
mapped_dataset = dataset.map(lambda x: tokenizer(x["sentence1"], x["sentence2"]), batched=True)



data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
train_args = TrainingArguments("test-run-1")

trainer = Trainer(
    model=model,
    args=train_args,
    train_dataset=mapped_dataset["train"],
    eval_dataset=mapped_dataset["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)
trainer.train()


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Step,Training Loss
500,0.530000
1000,0.287100


TrainOutput(global_step=1377, training_loss=0.3391527163436989, metrics={'train_runtime': 59.2521, 'train_samples_per_second': 185.715, 'train_steps_per_second': 23.24, 'total_flos': 405114969714960.0, 'train_loss': 0.3391527163436989, 'epoch': 3.0})

In [26]:
logits, labels, _  = trainer.predict(mapped_dataset["test"])
import numpy as np
predictions = np.argmax(logits, axis=-1)


In [53]:
all = np.sum(predictions == labels)
print(all/len(labels))

0.8417391304347827


In [78]:
bert = model.bert.encoder.layer[0].attention.self
print(type(bert))
for a in dir(bert):
    print(a)


<class 'transformers.models.bert.modeling_bert.BertSdpaSelfAttention'>
T_destination
__annotations__
__call__
__class__
__delattr__
__dict__
__dir__
__doc__
__eq__
__format__
__ge__
__getattr__
__getattribute__
__getstate__
__gt__
__hash__
__init__
__init_subclass__
__le__
__lt__
__module__
__ne__
__new__
__reduce__
__reduce_ex__
__repr__
__setattr__
__setstate__
__sizeof__
__str__
__subclasshook__
__weakref__
_apply
_backward_hooks
_backward_pre_hooks
_buffers
_call_impl
_compiled_call_impl
_forward_hooks
_forward_hooks_always_called
_forward_hooks_with_kwargs
_forward_pre_hooks
_forward_pre_hooks_with_kwargs
_get_backward_hooks
_get_backward_pre_hooks
_get_name
_is_full_backward_hook
_is_hf_initialized
_load_from_state_dict
_load_state_dict_post_hooks
_load_state_dict_pre_hooks
_maybe_warn_non_full_backward_hook
_modules
_named_members
_non_persistent_buffers_set
_parameters
_register_load_state_dict_pre_hook
_register_state_dict_hook
_replicate_for_data_parallel
_save_to_state_dict


TypeError: object of type 'NoneType' has no len()